## Vector Indexing Tutorial

### BM25

In [42]:
%%bash
pip install -q hnswlib

In [ ]:
import torch
from torch import nn
from torch.functional import tensordot
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.nn import CosineEmbeddingLoss
from torch import Tensor

from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import DPRContextEncoder
from typing import List, Dict

from tqdm import tqdm
import numpy as np
from datasets import Dataset
import hnswlib

### Neural Retrieval

Lets make it a huggingface dataset out of pure convenience

In [ ]:
imdb_dataset = Dataset.load_from_disk("imdb_top_10k_embeddings_dataset")
imdb_embeddings = np.load("imdb_top_10k_embeddings.npy")

### Dense Embedding

In [ ]:
class Transformer_embedder(nn.Module):
    def __init__(self, feat_extractor_name: str = ""):
        """Transformer Embedding model

        Args:
            feat_extractor_name (str, optional): Name of the feature extracator from HF hub or torch Hub.
        """
        super(Transformer_embedder, self).__init__()

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.feat_extractor_name = feat_extractor_name

        if "dpr" in feat_extractor_name.lower():
            feat_extractor = DPRContextEncoder.from_pretrained(feat_extractor_name)
        else:
            feat_extractor = AutoModel.from_pretrained(feat_extractor_name)

        self.tokenizer = AutoTokenizer.from_pretrained(feat_extractor_name)

        self.normalize = True
        self.feat_extractor = feat_extractor
        self.embeding_shape = self.get_extractor_output_shape()

    def get_extractor_output_shape(self):
        last_layer = list(self.feat_extractor.named_children())[-1]

        if hasattr(list(last_layer[1].modules())[1], "out_features"):
            shape = list(last_layer[1].modules())[1].out_features
        else:
            shape = self.feat_extractor.config.hidden_size

        return shape

    def mean_pooling(self, model_output: Tensor, attention_mask: Tensor):
        token_embeddings = model_output[
            0
        ]  # First element of model_output contains all token embeddings
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        )
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
            input_mask_expanded.sum(1), min=1e-9
        )

    def pool(self, embedding: Tensor, attention_mask: Tensor, pool_type: str = "mean"):

        if "mean" in pool_type:
            pooled = self.mean_pooling(embedding, attention_mask)
        else:
            pooled = embedding.last_hidden_state[:, 0, :]

        return pooled

    def __call__(
        self, input_ids: Tensor, attention_mask: Tensor, labels: Tensor = None, **kwargs
    ):

        # print('input_ids.shape: ', input_ids.shape)
        embedding = self.feat_extractor(input_ids, attention_mask)

        if "dpr" in self.feat_extractor_name.lower():
            pooled = embedding.pooler_output
        else:
            pooled = self.pool(embedding, attention_mask, pool_type="mean")
        # print('embedding.shape: ', embedding.last_hidden_state.shape)
        # last_hidden_states = embedding.last_hidden_state
        # print('last_hidden_states.shape: ', last_hidden_states.shape)
        # pooled = self.pool(last_hidden_states, attention_mask, pool_type='mean')
        # print('pooled.shape: ', pooled.shape)

        if self.normalize:
            pooled = F.normalize(pooled, p=2, dim=1)

        # print(pooled.shape)
        return pooled

In [ ]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"

# The following is a bigger model and might require slight modification in the code
# follow this link for more details: https://huggingface.co/intfloat/e5-mistral-7b-instruct
# model_ckpt = "intfloat/e5-mistral-7b-instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedder = Transformer_embedder(model_ckpt)
embedder = embedder.to(device)


def get_embeddings(text_list):
    encoded_input = embedder.tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    embedder.eval()
    with torch.inference_mode():
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
        model_output = embedder(**encoded_input)
    return model_output

In [ ]:
question = "Armenia"
question_embedding = get_embeddings([question]).cpu().detach().numpy()[0]
question_embedding.shape

## Vector Indexing

### Inverted File Indexing

Inverted file indexing is a technique used to index documents in a corpus. It is a data structure that maps terms to the documents that contain them. The inverted file index is a dictionary where the keys are terms and the values are lists of document IDs. The inverted file index is used to quickly find documents that contain a given term.

Mathematically, the inverted file index is defined as follows:

$$
I = \{t_1: [d_1, d_2, \ldots, d_n], t_2: [d_1, d_2, \ldots, d_n], \ldots, t_m: [d_1, d_2, \ldots, d_n]\}
$$

Where $I$ is the inverted file index, $t_i$ is the $i$-th term, and $d_i$ is the $i$-th document ID.





In [ ]:
from sklearn.cluster import MiniBatchKMeans


class IVFFlatIndexer:
    def __init__(self, n_clusters: int = 100, n_init: int = 10, max_iter: int = 300):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.index = MiniBatchKMeans(
            n_clusters=self.n_clusters,
            n_init=self.n_init,
            max_iter=self.max_iter,
            init_size=3 * self.n_clusters,
        )

    def fit(self, X):
        self.index.fit(X)

    def search(self, X, top_k=5):
        return self.index.predict(X)

    def get_cluster_centers(self):
        return self.index.cluster_centers_

    def get_cluster_labels(self):
        return self.index.labels_


indexer = IVFFlatIndexer(n_clusters=100)
indexer.fit(imdb_embeddings)

print(indexer.search(question_embedding.reshape(1, -1)))


def get_top_k_similar(question_embedding, embeddings, indexer, k=5):
    cluster_id = indexer.search(question_embedding.reshape(1, -1))
    cluster_embeddings = embeddings[indexer.get_cluster_labels() == cluster_id]
    distances = np.dot(cluster_embeddings, question_embedding)
    top_k_indices = np.argsort(distances)[::-1][:k]
    return top_k_indices


top_k_indices = get_top_k_similar(question_embedding, imdb_embeddings, indexer, k=5)
top_k_indices

print("Question: ", question)
print("Top 5 similar movies:")
print("=" * 100)
for idx in top_k_indices:
    print(imdb_dataset[idx.item()]["Movie Name"])
    print(imdb_dataset[idx.item()]["Plot"])
    print("=" * 100)

### Locality Sensitive Hashing

Local sensitive hashing (LSH) is a technique used to find similar items in a large dataset. LSH is used to reduce the dimensionality of the data and to find similar items in the reduced space. LSH is used in many applications, such as near-duplicate detection, recommendation systems, and clustering.

Mathematically, LSH is defined as follows:

$$
h(x) = \left\{
\begin{array}{ll}
1 & \text{if } x \geq t \\
0 & \text{if } x < t
\end{array}
\right.
$$

Where $h(x)$ is the hash function, $x$ is the input value, and $t$ is the threshold value.

In [ ]:
class LSHIndexer:
    def __init__(self, n_bits: int = 8):
        self.n_bits = n_bits

    def fit(self, X):
        self.index = np.packbits(X > 0, axis=1)

    def search(self, X, top_k=5):
        query = np.packbits(X > 0, axis=1)
        distances = np.dot(query, self.index.T)
        top_k_indices = np.argsort(distances)[::-1][:top_k]
        return top_k_indices

    def get_index(self):
        return self.index


indexer = LSHIndexer(n_bits=8)
indexer.fit(imdb_embeddings)

top_k_indices = indexer.search(question_embedding.reshape(1, -1))
top_k_indices = top_k_indices[0][:5]


print("Question: ", question)
print("Top 5 similar movies:")
print("=" * 100)
for idx in top_k_indices:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("=" * 100)

### Product Quantization

Product quantization is a technique used to compress high-dimensional vectors into low-dimensional vectors. Product quantization is used to reduce the storage and computation costs of working with high-dimensional vectors. Product quantization is used in many applications, such as image retrieval, text search, and recommendation systems.

Mathematically, product quantization is defined as follows:

$$
Q(x) = \sum_{i=1}^{n} c_i \cdot q_i(x)
$$

Where $Q(x)$ is the quantized vector, $x$ is the input vector, $c_i$ is the $i$-th codebook vector, and $q_i(x)$ is the $i$-th quantization function.

In [ ]:
class ProductQuantization:
    def __init__(self, sentence_embeddings, n_subvectors=8):
        """
        Product Quantization Index

        Args:
            sentence_embeddings (np.array): Array of sentence embeddings
            n_subvectors (int): Number of subvectors to divide each embedding into
        """
        # Subspace dimensionality
        self.d = sentence_embeddings.shape[1] // n_subvectors
        # Number of subvectors
        self.m = n_subvectors

        # Generate random projection matrix
        self.R = np.random.randn(self.d * self.m, self.d)
        self.R /= np.linalg.norm(self.R, axis=0)

        # Project sentence embeddings onto subspaces
        self.subspace_embeddings = np.dot(sentence_embeddings, self.R)

        # Quantize subspace embeddings
        self.quantized_subspace_embeddings = np.round(self.subspace_embeddings / self.d)

    def search(self, query_embedding, top_k=1):
        """
        Search for the most similar sentences to a query embedding

        Args:
            query_embedding (np.array): Query embedding
            top_k (int): Number of similar sentences to return

        Returns:
            list: List of indices of similar sentences
        """
        # Project query embedding onto subspaces
        query_subspace_embeddings = np.dot(query_embedding, self.R)

        # Quantize query subspace embeddings
        quantized_query_subspace_embeddings = np.round(
            query_subspace_embeddings / self.d
        )

        # Find nearest neighbors in each subspace
        nearest_neighbors = []
        for i in range(self.m):
            distances = np.linalg.norm(
                self.quantized_subspace_embeddings[:, i]
                - quantized_query_subspace_embeddings[i],
                ord=2,
            )
            nearest_neighbors.append(np.argsort(distances)[:top_k])

        # Combine nearest neighbors from each subspace
        combined_nearest_neighbors = set()
        for neighbors in nearest_neighbors:
            combined_nearest_neighbors.update(neighbors)

        return list(combined_nearest_neighbors)


product_quantizer = ProductQuantization(imdb_embeddings, n_subvectors=16)
top_k_indices = product_quantizer.search(question_embedding, top_k=5)
print(top_k_indices)

print("Question: ", question)
print("Top 5 similar movies:")
print("=" * 100)
for idx in top_k_indices:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("=" * 100)

Navigable Small World Graphs

Navigable small world graphs are a technique used to build a graph that can be used to efficiently search for similar items in a large dataset. Navigable small world graphs are used to reduce the search time and space required to find similar items in a large dataset. 

Mathematically, navigable small world graphs are defined as follows:

$$
G = (V, E)
$$

Where $G$ is the navigable small world graph, $V$ is the set of vertices, and $E$ is the set of edges.

The algorithm for building a navigable small world graph is as follows:

1. Initialize the graph with a single vertex.
2. Add a new vertex to the graph.
3. Connect the new vertex to the nearest vertex in the graph.
4. Repeat steps 2 and 3 until all vertices are connected.


In [ ]:
class HNSWIndexer:

    def __init__(self, sentence_embeddings, ef=50, M=16):
        """
        HNSW Index

        Args:
            sentence_embeddings (np.array): Array of sentence embeddings
            ef (int): Number of neighbors to inspect during search
            M (int): Number of neighbors to keep in graph
        """
        self.ef = ef
        self.M = M
        self.index = hnswlib.Index(space="cosine", dim=sentence_embeddings.shape[1])
        self.index.init_index(
            max_elements=sentence_embeddings.shape[0], ef_construction=self.ef, M=self.M
        )
        self.index.add_items(sentence_embeddings)

    def search(self, query_embedding, top_k=1):
        """
        Search for the most similar sentences to a query embedding

        Args:
            query_embedding (np.array): Query embedding
            top_k (int): Number of similar sentences to return

        Returns:
            list: List of indices of similar sentences
        """
        self.index.set_ef(self.ef)
        labels, distances = self.index.knn_query(query_embedding, k=top_k)
        return labels


hnsw_indexer = HNSWIndexer(imdb_embeddings)
top_k_indices = hnsw_indexer.search(question_embedding, top_k=5)
print(top_k_indices)

print("Question: ", question)
print("Top 5 similar movies:")
print("=" * 100)

for idx in top_k_indices[0]:
    idx = idx.item()
    print(imdb_dataset[idx]["Movie Name"])
    print(imdb_dataset[idx]["Plot"])
    print("=" * 100)